<p style="font-family: Arial; font-size:2.75em;color:blue; font-style:bold"><br>
Projet 6 :<br><br>
Catégorisez automatiquement des questions<br><br>
</p><br>

* **Data Source / Data Source :** 
    - Data :
        - Outils d'extraction de data : https://data.stackexchange.com/stackoverflow
        - Script : https://github.com/EricJacquesPro/TextCategorization/blob/master/SQL/ExtractionDonnees.sql
    - Source Python : https://github.com/EricJacquesPro/TextCategorization
* **Description / Description : ** Projet 6 de la formation Data Scientist Proposée par OpenClassroom
    - Analyse LDA
    - Entrainement LDA
* **Auteur / Author : ** Eric JACQUES 
* **Date : ** 18/07/2020 

In [1]:
import sys
import os
import numpy as np
import pandas as pd
sys.path.insert(0, './Python/')

In [2]:
'''
python class for the data engineering (reading, cleaning, training...)
localised in ./Python
'''
from tagText import TagText 

C:\Users\naru_\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [ ]:
tagText = TagText()
print(tagText.urlDirectory)#folder for cvs file generated by SQL query (cf. I) 
print(tagText.fileName)#name of the cvs file generated by SQL query (cf. I)
data_question = tagText.read_source()
data_question.head()

tagText.nombre_post_entree = 50000
tagText.precision = 50000

Data/
QuestionVsTags.csv


In [ ]:
data_preprocessed = data_question['body'].apply(tagText.preprocessing)
print(data_preprocessed[1:9])

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
III.2 - Data - Bag of words - TF-IDF</p>

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
IV.2-a - Data - Tag generator - Unsupervised - LDA</p>

In [ ]:
perfsTrain, perfsValidation = tagText.lda_find_topic_number(
                    data_preprocessed,
                    5,
                    81,
                    2
                )


In [ ]:
print(len(perfsTrain))
print(len(perfsValidation))


In [ ]:
print(max(perfsTrain))
print(max(perfsValidation))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
mini=min(perfsTrain)
#print(perfs)
#print(perfs-mini)
x = range(5,81,2)

from sklearn.preprocessing import scale
perfs_normalized = scale( perfsTrain, axis=0, with_mean=True, with_std=True, copy=True )
x_normalized = scale( x, axis=0, with_mean=True, with_std=True, copy=True )

plt.bar(x, perfsTrain)
plt.ylabel("Performance")
plt.xlabel("number of topics")
plt.title("performance by number of topics")
plt.show()

plt.bar(x, perfs_normalized)
plt.ylabel("Performance normalized")
plt.xlabel("number of topics")
plt.title("performance normalized by number of topics")
plt.show()

perfs_topic_optimized = np.add(x_normalized, perfs_normalized)
plt.bar(x, perfs_topic_optimized)
plt.ylabel("Performance normalized + number of topic normalized")
plt.xlabel("number of topics")
plt.title("performance optimized by number of topics")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
mini=min(perfsValidation)
#print(perfs)
#print(perfs-mini)
x = range(5,81,2)

from sklearn.preprocessing import scale
perfs_normalized = scale( perfsValidation, axis=0, with_mean=True, with_std=True, copy=True )
x_normalized = scale( x, axis=0, with_mean=True, with_std=True, copy=True )

plt.bar(x, perfsValidation)
plt.ylabel("Performance")
plt.xlabel("number of topics")
plt.title("performance by number of topics")
plt.show()

plt.bar(x, perfs_normalized)
plt.ylabel("Performance normalized")
plt.xlabel("number of topics")
plt.title("performance normalized by number of topics")
plt.show()

perfs_topic_optimized = np.add(x_normalized, perfs_normalized)
plt.bar(x, perfs_topic_optimized)
plt.ylabel("Performance normalized + number of topic normalized")
plt.xlabel("number of topics")
plt.title("performance optimized by number of topics")
plt.show()

In [ ]:
    def lda_predict(
        model,
        text,
        lda,
        lda_df_topic_keywords,
        lda_tf_vectorizer,
        no_top_words
    ):
        '''
        predict tag form text in function of lda, topic ad tf vectorizer
        '''
        text = [text]
        mytext = lda_tf_vectorizer.transform(text)
        lda_topic_probability_scores = lda.transform(mytext)
        lda_topic = lda_df_topic_keywords.iloc[
            tagText.np.argmax(lda_topic_probability_scores),
            :
        ].values.tolist()
        topic_array = model.np.array(lda_topic)
        lda_feature_names = lda_tf_vectorizer.get_feature_names()
        """
        print(lda_topic)
        print(topic_array.argsort())
        print(lda_feature_names)
        """
        return (
            " ".join(
                        [
                            lda_feature_names[i]
                            for i in topic_array.argsort()
                            [
                                :-no_top_words - 1:-1
                            ]
                        ]
                )
        )

In [ ]:
lda_tagText, df_topic_keywords, tf_vectorizer = tagText.lda_prepare_tag(data_preprocessed, 32)

print(lda_predict(tagText, 'I cannot intall a git repository. I try github and gitlab', lda_tagText, df_topic_keywords, tf_vectorizer, 5))
print(tagText.lda_predict( "How to cast number as integer in java", lda_tagText, df_topic_keywords, tf_vectorizer, 5))

In [ ]:

print(lda_predict(tagText, 'I cannot intall a git repository. I try github and gitlab', lda_tagText, df_topic_keywords, tf_vectorizer, 5))

In [ ]:

print(tagText.lda_predict( "How to cast number as integer in java", lda_tagText, df_topic_keywords, tf_vectorizer, 5))

In [ ]:

print(tagText.lda_predict( "How to cast number as integer in java", lda_tagText, df_topic_keywords, tf_vectorizer, 10))

In [ ]:
from joblib import dump, load
dump(lda_tagText, 'lda.joblib',0)
dump(df_topic_keywords, 'df_lda_topic_keywords.joblib',0)
dump(tf_vectorizer, 'tf_lda_vectorizer.joblib',0)

In [ ]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic :", topic_idx, ":" )
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 5
print('Topics : ')
display_topics(lda_tagText, tf_vectorizer.get_feature_names(), no_top_words)



In [ ]:

print('Topics From working class : ')
tagText.display_topics(lda_tagText, tf_vectorizer.get_feature_names(), no_top_words)